# Spleen Dataset
http://medicaldecathlon.com/

In [1]:
# Imports
import json
import os
import torch
import shutil
import nibabel as nib
import numpy as np
from skimage.transform import resize
import matplotlib.pyplot as plt
from skimage.io import imsave
import random
import imageio
import cv2


# Data Loading
- Set base_dir
- Make sure to resize the image and then set the max images because its gonna take too long.
set to 

In [3]:
    
def load_nifti_image(file_path):
    return nib.load(file_path).get_fdata()

def split_train_test(data, max_train_images, max_test_images, seed):
    random.seed(seed)
    random.shuffle(data)

    train_data = data[:max_train_images]
    test_data = data[max_train_images:max_train_images + max_test_images]
    finetune_data = data[max_train_images + max_test_images:]

    return train_data, test_data, finetune_data


def save_images_and_labels(data, base_dir, img_dir, lbl_dir, image_rows, image_cols, prefix):
    os.makedirs(img_dir, exist_ok=True)
    os.makedirs(lbl_dir, exist_ok=True)
    
    for idx, entry in enumerate(data):
        img_path = os.path.join(base_dir, entry['image'])
        lbl_path = os.path.join(base_dir, entry['label'])
        
        img = load_nifti_image(img_path)
        lbl = load_nifti_image(lbl_path)
        
        for i in range(img.shape[2]):
            img_2d = img[:, :, i]
            lbl_2d = lbl[:, :, i]
            
            img_2d = resize(img_2d, (image_rows, image_cols), preserve_range=True)
            lbl_2d = resize(lbl_2d, (image_rows, image_cols), preserve_range=True)
            
            img_2d = (img_2d - np.min(img_2d)) / (np.max(img_2d) - np.min(img_2d)) * 255
            lbl_2d = (lbl_2d - np.min(lbl_2d)) / (np.max(lbl_2d) - np.min(lbl_2d)) * 255
            
            img_2d = img_2d.astype(np.uint8)
            lbl_2d = lbl_2d.astype(np.uint8)
            
            img_filename = f'{prefix}_img_{idx:04d}_{i:03d}.png'
            lbl_filename = f'{prefix}_img_{idx:04d}_{i:03d}.png'
            
            img_save_path = os.path.join(img_dir, img_filename)
            lbl_save_path = os.path.join(lbl_dir, lbl_filename)
            
            imageio.imwrite(img_save_path, img_2d)
            imageio.imwrite(lbl_save_path, lbl_2d)


In [ ]:
base_dir = 'spleen_data/Task09_Spleen'

json_path = os.path.join(base_dir, 'dataset.json')
with open(json_path, 'r') as f:
    data_json = json.load(f)

image_rows = 512
image_cols = 512

max_train_images = 20
max_test_images = 5
seed = 0

train_data, test_data, finetune_data = split_train_test(data_json['training'], max_train_images, max_test_images, seed)

current_dir = os.getcwd()
train_img_dir = os.path.join(current_dir, 'images/train')
train_lbl_dir = os.path.join(current_dir, 'mask/train')
test_img_dir = os.path.join(current_dir, 'images/val')
test_lbl_dir = os.path.join(current_dir, 'mask/val')
finetune_img_dir = os.path.join(current_dir, 'images/finetune')
finetune_lbl_dir = os.path.join(current_dir, 'mask/finetune')



# mask_dir = os.path.join(current_dir, 'mask')
# img_dir = os.path.join(current_dir, 'images')
# if os.path.exists(mask_dir) and os.path.isdir(mask_dir):
#     shutil.rmtree(mask_dir)

# if os.path.exists(img_dir) and os.path.isdir(img_dir):
#     shutil.rmtree(img_dir)


# save_images_and_labels(train_data, base_dir, train_img_dir, train_lbl_dir, image_rows, image_cols, 'train')

# save_images_and_labels(test_data, base_dir, test_img_dir, test_lbl_dir, image_rows, image_cols, 'test')

save_images_and_labels(finetune_data, base_dir, finetune_img_dir, finetune_lbl_dir, image_rows, image_cols, 'finetune')

# Segmentation

In [4]:
def createmask(input_dir, output_dir):
    
    
    os.makedirs(output_dir, exist_ok=True)

    
    for j in os.listdir(input_dir):
        image_path = os.path.join(input_dir, j)
        mask = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        _, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

        H, W = mask.shape
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        polygons = []
        for cnt in contours:
            if cv2.contourArea(cnt) > 1:
                polygon = []
                for point in cnt:
                    x, y = point[0]
                    polygon.append(x / W)
                    polygon.append(y / H)
                polygons.append(polygon)

        with open('{}.txt'.format(os.path.join(output_dir, j)[:-4]), 'w') as f:
            for polygon in polygons:
                for p_, p in enumerate(polygon):
                    if p_ == len(polygon) - 1:
                        f.write('{}\n'.format(p))
                    elif p_ == 0:
                        f.write('0 {} '.format(p))
                    else:
                        f.write('{} '.format(p))

            f.close()

In [5]:
input_train_masks = 'mask/train'
output_train_masks = 'labels/train'
input_test_masks = 'mask/val'
output_test_masks = 'labels/val'

current_dir = os.getcwd()
label_dir = os.path.join(current_dir, 'labels')
if os.path.exists(label_dir) and os.path.isdir(label_dir):
    shutil.rmtree(label_dir)

createmask(input_train_masks, output_train_masks)
createmask(input_test_masks, output_test_masks)

In [1]:
import torch

print(torch.__version__)
if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")
    
    for i in range(num_gpus):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
    
    device = 'cuda:0'
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = 'cpu'
    print("CUDA is not available. Training on CPU.")

2.1.1+cu121
Number of GPUs available: 1
GPU 0: NVIDIA GeForce RTX 2070 SUPER
Using GPU: NVIDIA GeForce RTX 2070 SUPER


# Running The Experiment.

In [ ]:
from ultralytics import YOLO
# https://docs.ultralytics.com/usage/cfg/#train

model = YOLO('yolov8n-seg.pt')  
# model = YOLO('yolov8s-p2.yaml').load('yolov8s.pt')  
 
# model.train(data='config.yaml', epochs=100, imgsz=640, seed=1, lr0=0.001, lrf=0.00005, close_mosaic=25)
# Make Epochs 10, Mosaic = less than number of epochs, at least 10% of the epochs.
# model.train(data='config.yaml', epochs=50, imgsz=640, seed=1, optimizer="Adam", lr0=0.001, lrf=0.00005, close_mosaic= 45)
model.train(data='original.yaml', epochs=30, imgsz=640, seed=1, optimizer="Adam", lr0=0.001, lrf=0.00005, close_mosaic=10)

# model.train(data='config.yaml', epochs=20, imgsz=640, seed=1, lr0=0.0001, lrf=0.000005, close_mosaic=10)
# model.train(data='config.yaml', epochs=20, imgsz=640, seed=1)

In [2]:
from ultralytics import YOLO 

def diffmodel(yaml, path = 'runs/segment/train13/weights/best.pt'):
    model = YOLO(path)  # or 'yolov8s.pt', 'yolov8m.pt', 'yolov8l.pt', 'yolov8x.pt'

    results = model.val(data=yaml, imgsz=640)


    print("Class indices with average precision:", results.ap_class_index)
    print("Average precision for all classes:", results.box.all_ap)
    print("Mean average precision at IoU=0.50:", results.box.map50)
    print("Mean recall:", results.box.mr)

    precision = results.box.mp
    recall = results.box.mr

    f1_score = 2 * (precision * recall) / (precision + recall)

    print(f"Name: {yaml}")
    print(f"F1 Score: {f1_score}")

In [5]:
diffmodel('blurred30x.yaml')

Ultralytics YOLOv8.2.8  Python-3.10.5 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 8192MiB)
YOLOv8n-seg summary (fused): 195 layers, 3258259 parameters, 0 gradients, 12.0 GFLOPs


val: Scanning C:\Users\Ben\Documents\WorkComp\Uoa Engineering Master\Year 4\Sem2\760\Project\compsci-760-project\doubleblur_noise_denoise\labels\val.cache... 327 images, 232 backgrounds, 0 corrupt: 100%|██████████| 327/327 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 21/21 [00:04<00:00,  4.34it/s]


                   all        327         96      0.702      0.344      0.373      0.195      0.637      0.302      0.341      0.142
Speed: 0.5ms preprocess, 6.9ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs\segment\val61
Class indices with average precision: [0]
Average precision for all classes: [[    0.37317     0.34901     0.30776     0.27921     0.24342     0.16897     0.13303    0.069082    0.026922   0.0011092]]
Mean average precision at IoU=0.50: 0.37316565433870114
Mean recall: 0.34375
Name: blurred30x.yaml
F1 Score: 0.4615089049984497


In [4]:
train_models('blurred30x.yaml')


Training model with configuration from: blurred30x.yaml
New https://pypi.org/project/ultralytics/8.3.17 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.8  Python-3.10.5 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 8192MiB)
engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=blurred30x.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train27, exist_ok=False, pretrained=True, optimizer=Adam, verbose=True, seed=1, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=Fal

train: Scanning C:\Users\Ben\Documents\WorkComp\Uoa Engineering Master\Year 4\Sem2\760\Project\compsci-760-project\doubleblur_noise_denoise\labels\train... 1921 images, 1325 backgrounds, 0 corrupt: 100%|██████████| 1921/1921 [00:08<00:00, 232.04it/s]


train: New cache created: C:\Users\Ben\Documents\WorkComp\Uoa Engineering Master\Year 4\Sem2\760\Project\compsci-760-project\doubleblur_noise_denoise\labels\train.cache


C:\Users\Ben\AppData\Roaming\Python\Python310\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.14). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...


      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


      30/30      2.68G      1.036      1.497     0.6468      1.149          0        640: 100%|██████████| 121/121 [00:21<00:00,  5.57it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:02<00:00,  4.50it/s]


                   all        327         96      0.922      0.862      0.922       0.61      0.933      0.872      0.925      0.587

30 epochs completed in 0.243 hours.
Optimizer stripped from runs\segment\train27\weights\last.pt, 6.8MB
Optimizer stripped from runs\segment\train27\weights\best.pt, 6.8MB

Validating runs\segment\train27\weights\best.pt...
Ultralytics YOLOv8.2.8  Python-3.10.5 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2070 SUPER, 8192MiB)
YOLOv8n-seg summary (fused): 195 layers, 3258259 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 11/11 [00:04<00:00,  2.75it/s]


                   all        327         96      0.916      0.875      0.922      0.609      0.927      0.885      0.925      0.589
Speed: 0.5ms preprocess, 3.4ms inference, 0.0ms loss, 3.6ms postprocess per image
Results saved to runs\segment\train27
Training complete for: blurred30x.yaml
F1 Score not found for blurred30x.yaml. Check results format.


In [3]:
from ultralytics import YOLO 
import logging


def train_models(yaml_file, model_name='yolov8n-seg.pt', epochs=30, imgsz=640, seed=1, optimizer="Adam", lr0=0.001, lrf=0.00005, close_mosaic=10):
    model = YOLO(model_name)
    
    try:
        print(f"\nTraining model with configuration from: {yaml_file}")
        
        results = model.train(
            data=yaml_file,  
            epochs=epochs,  
            imgsz=imgsz,
            seed=seed,
            optimizer=optimizer, 
            lr0=lr0,
            lrf=lrf,
            close_mosaic=close_mosaic 
        )
        
        print(f"Training complete for: {yaml_file}")
        print(f"F1 Score not found for {yaml_file}. Check results format.")
    
    except Exception as e:

        print(f"Error occurred while training with {yaml_file}: {e}")


In [ ]:
train_models('blurred30x.yaml')

In [ ]:
train_models('denoised.yaml')

In [ ]:
train_models('doubleblurunblur_ft.yaml')

In [ ]:
train_models('doubleblur.yaml')

In [ ]:
train_models('doubleblurunblur.yaml')

In [ ]:
train_models('lowres.yaml')

In [ ]:
train_models('lowresx4.yaml')

In [ ]:
train_models('lowresx8.yaml')

In [ ]:
train_models('noise.yaml')

In [ ]:
train_models('original_upscaled.yaml')

In [ ]:
# Relevant ones.
filter_non_empty_masks('doubleblur')
filter_non_empty_masks('doubleblur_deblurred')
filter_non_empty_masks('noisy_ft_denoised')
filter_non_empty_masks('noisy')
filter_non_empty_masks('lowres')
filter_non_empty_masks('lowresx4')
filter_non_empty_masks('lowresx8')

In [ ]:
# diffmodel('doubleblur.yaml','runs/segment/doubleblur/weights/best.pt')
# diffmodel('doubleblurunblur.yaml','runs/segment/doubleunblur/weights/best.pt')
# diffmodel('denoised_ft.yaml','runs/segment/denoised_ft/weights/best.pt')
# diffmodel('noise.yaml','runs/segment/noise/weights/best.pt')
# diffmodel('lowres.yaml','runs/segment/lowres/weights/best.pt')
# diffmodel('lowresx4.yaml','runs/segment/lowresx4/weights/best.pt')
# diffmodel('lowresx8.yaml','runs/segment/lowresx8/weights/best.pt')

In [ ]:

import torch
from ultralytics import YOLO

diffmodel('blur.yaml','runs/segment/train13/weights/best.pt')
diffmodel('blurunblur.yaml')
diffmodel('denoised_ft.yaml')
diffmodel('denoised.yaml')
diffmodel('doubleblurunblur_ft.yaml')
diffmodel('doubleblur.yaml')
diffmodel('doubleblurunblur.yaml')
diffmodel('lowres.yaml')
diffmodel('lowresx4.yaml')
diffmodel('lowresx8.yaml')
diffmodel('noise.yaml')
diffmodel('original_upscaled.yaml')
diffmodel('original.yaml')

In [ ]:
diffmodel('original.yaml')